# Emittance Optimization with BAX (Differential evolution)

# Imports

BEFORE RUNNING, NEED TO:
- replace variables with emittance variables
- define meas_dim
- define scale factor
- verify image constraints/screen parameters

In [1]:
import sys
from AWAControl.badger.environments.awa import Environment
awa_env = Environment()
awa_env.screen_name = "DYG5"
awa_env.image_diagnostic.apply_bounding_box_constraint = False 

ModuleNotFoundError: No module named 'AWAControl'

In [ ]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

def evaluate_function(inputs: dict) -> dict:
    # caput valueslog_transform_acquisition_function
    awa_env.set_variables(inputs)

    # wait for changes to occur - use small wait time for interpolated measurements
    time.sleep(1)

    results = awa_env.get_observables(["total_size"])
    results['Sx_sq'] = results["Sx"]**2
    results['Sy_sq'] = results["Sy"]**2
    return results

evaluator = Evaluator(function=evaluate_function)

In [ ]:
# test beam measurement
evaluator.evaluate({"DQ4":0.0,"DQ5":0.0,"DQ6":0.0,"DT8H":0.0,"DT9H":0.0,"DT8V":0.0,"DT9V":0.0})

# Construct vocs

In [ ]:
from xopt import VOCS
from xopt import Xopt

from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from gpytorch.priors.torch_priors import GammaPrior
from xopt.generators.bayesian.bax_generator import BaxGenerator
from xopt.generators.bayesian.bayesian_exploration import BayesianExplorationGenerator
from copy import deepcopy
import pandas as pd
import numpy as np

IMAGE_CONSTRAINTS = {
                "log10_total_intensity": ["GREATER_THAN", 6.5]
            }

vocs = VOCS(
    variables = {
        "DQ4":[-0.75,0.75],
        "DQ6":[-0.75,0.75],
        "DT8V":[-3.1,3.1],
        "DT9V":[-3.1,3.1] 
    },
    constraints = IMAGE_CONSTRAINTS,
    observables = ["Sx_sq", "Sy_sq"],
)

meas_dim = 0

# Prepare generator options.
In this example, we use a specialty covariance module (Matern x Quadratic kernel) for our beam size model.

In [ ]:
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from gpytorch.priors.torch_priors import GammaPrior

from xopt.generators.bayesian.models.standard import StandardModelConstructor
from xopt.generators.bayesian.bax_generator import BaxGenerator

# prepare custom covariance module
tuning_dims = list(range(vocs.n_variables))
tuning_dims.remove(meas_dim)
covar_module_x = (MaternKernel(ard_num_dims=len(tuning_dims), 
                              active_dims=tuning_dims, 
                              lengthscale_prior=None) * 
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )

scaled_covar_module_x = ScaleKernel(covar_module_x)
covar_module_y = (MaternKernel(ard_num_dims=len(tuning_dims), 
                              active_dims=tuning_dims, 
                              lengthscale_prior=None) * 
                              PolynomialKernel(power=2, active_dims=[meas_dim])
                 )
scaled_covar_module_y =  ScaleKernel(covar_module_y)

# prepare options for Xopt generator
covar_module_dict = {'Sx_sq': scaled_covar_module_x,
                     'Sy_sq': scaled_covar_module_y}

model_constructor = StandardModelConstructor(covar_modules=covar_module_dict, use_low_noise_prior=True)

In [ ]:
from xopt.numerical_optimizer import LBFGSOptimizer, GridOptimizer
numerical_optimizer = LBFGSOptimizer(
                                    n_restarts=20,
                                    max_time=2)
# numerical_optimizer = GridOptimizer()

# Construct generator, evaluator, Xopt objects

In [ ]:
from emitopt.algorithms import DifferentialEvolutionEmitBmag

#Prepare Algorithm
algo_kwargs = {
        'x_key': 'Sx_sq',
        'y_key': 'Sy_sq',
        'scale_factor': .00893,
        'q_len': 0.12,
        'rmat_x': torch.tensor([[1., 2.06],
                                [0, 1.]]),
        'rmat_y': torch.tensor([[1., 2.06],
                                [0, 1.]]),
        'n_samples': 10,
        'meas_dim': meas_dim,
        'n_steps_measurement_param': 7,
        'thick_quad': True,
        'use_bmag': False,
}
algo = DifferentialEvolutionEmitBmag(**algo_kwargs)

#construct BAX generator
generator = BaxGenerator(vocs=vocs, 
                         gp_constructor=model_constructor, 
                         numerical_optimizer=numerical_optimizer,
                         algorithm=algo,
                         n_interpolate_points=5)

#construct evaluator
evaluator = Evaluator(function=measure_beamsize)

#construct Xopt optimizer
optimizer = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)

# Optimize

In [ ]:
# call X.random_evaluate() to generate random initial points and evaluate on test_func
optimizer.random_evaluate(5)

In [ ]:
from xopt.generators.bayesian.visualize import visualize_generator_model
optimizer.generator.train_model()
visualize_generator_model(optimizer.generator, 
                          variable_names=['x0','x1'], 
                            reference_point={'x0':torch.tensor([0.]),
                                             'x1':torch.tensor([0.]),
                                            'x2':torch.tensor([0.]),
                                             'x3':torch.tensor([0.]),
                                            },
                          show_acquisition=False)
visualize_generator_model(optimizer.generator, 
                          variable_names=['x1'], 
                            reference_point={'x0':torch.tensor([0.]),
                                             'x1':torch.tensor([0.]),
                                            'x2':torch.tensor([0.]),
                                             'x3':torch.tensor([0.]),
                                            },
                          show_acquisition=False)

In [ ]:
for i in range(10):
    print(i)
    start = time.time()
    optimizer.step()
    print(time.time() - start)

In [ ]:
from xopt.generators.bayesian.bax.visualize import visualize_virtual_objective
model = optimizer.generator.train_model()

# for i in range(10):
fig, ax = visualize_virtual_objective(optimizer.generator, 
                            variable_names=['x0'],
                            reference_point={'x0':torch.tensor([0.]),
                                             'x1':torch.tensor([0.]),
                                            'x2':torch.tensor([0.]),
                                             'x3':torch.tensor([0.]),
                                            },
                            n_grid=100,
                            n_samples=100,
                            # show_samples=True,
                            kwargs={'use_bmag':False},
                                     )